## SF crime data analysis

**In this notebook, I use Spark SQL for big data analysis on SF crime data from 2003 to 2018/05/15.** 

(https://data.sfgov.org/Public-Safety/Police-Department-Incident-Reports-Historical-2003/tmnf-yvry). 

The current version is OLAP (Online Analytical Processing) for SF crime data analysis.  

(--Possible plans in the future:
1. unsupervised learning for spatial data analysis.  
2. time series data analysis.  )

In [0]:
from csv import reader
from pyspark.sql import Row 
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import warnings

import os
os.environ["PYSPARK_PYTHON"] = "python3"


In [0]:
# download data from SF gov website

#import urllib.request
#urllib.request.urlretrieve("https://data.sfgov.org/api/views/tmnf-yvry/rows.csv?accessType=DOWNLOAD", "/tmp/myxxxx.csv")
#dbutils.fs.mv("file:/tmp/myxxxx.csv", "dbfs:/laioffer/spark_hw1/data/sf_03_18.csv")
#display(dbutils.fs.ls("dbfs:/laioffer/spark_hw1/data/"))
## link
# https://data.sfgov.org/api/views/tmnf-yvry/rows.csv?accessType=DOWNLOAD


In [0]:
data_path = "dbfs:/laioffer/spark_hw1/data/sf_03_18.csv"
# use this file name later

In [0]:

from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("crime analysis") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

sf_crime = spark.read.format("csv")\
  .option("header", "true")\
  .option("interSchema", "true")\
  .load(data_path)

#original dataset
display(sf_crime)


PdId,IncidntNum,Incident Code,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,location,SF Find Neighborhoods 2 2,Current Police Districts 2 2,Current Supervisor Districts 2 2,Analysis Neighborhoods 2 2,DELETE - Fire Prevention Districts 2 2,DELETE - Police Districts 2 2,DELETE - Supervisor Districts 2 2,DELETE - Zip Codes 2 2,DELETE - Neighborhoods 2 2,DELETE - 2017 Fix It Zones 2 2,Civic Center Harm Reduction Project Boundary 2 2,Fix It Zones as of 2017-11-06 2 2,DELETE - HSOC Zones 2 2,Fix It Zones as of 2018-02-07 2 2,"CBD, BID and GBD Boundaries as of 2017 2 2","Areas of Vulnerability, 2016 2 2",Central Market/Tenderloin Boundary 2 2,Central Market/Tenderloin Boundary Polygon - Updated 2 2,HSOC Zones as of 2018-06-05 2 2,OWED Public Spaces 2 2,Neighborhoods 2
3114751606302,031147516,06302,LARCENY/THEFT,PETTY THEFT FROM A BUILDING,Sunday,09/28/2003,10:00,SOUTHERN,NONE,1ST ST / BRANNAN ST,-120.5,90,POINT (-120.50000000000001 90),null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
5069701104134,050697011,04134,ASSAULT,BATTERY,Wednesday,06/22/2005,12:20,NORTHERN,NONE,1200 Block of EDDY ST,-122.428223303176,37.7818959488603,POINT (-122.42822330317601 37.7818959488603),97,4,11,39,7,9,11,29490,41,null,null,null,null,null,null,2,null,null,null,null,97
6074729204104,060747292,04104,ASSAULT,ASSAULT,Saturday,07/15/2006,00:55,CENTRAL,NONE,1600 Block of POWELL ST,-122.410672425337,37.799788690123,POINT (-122.41067242533701 37.799788690123),106,6,3,6,3,1,10,308,4,null,null,null,null,null,null,2,null,null,null,null,106
7103536315201,071035363,15201,ASSAULT,STALKING,Tuesday,09/25/2007,00:01,TARAVAL,NONE,400 Block of ULLOA ST,-122.458226300605,37.7413616001449,POINT (-122.458226300605 37.7413616001449),49,10,8,41,1,8,4,59,40,null,null,null,null,null,null,1,null,null,null,null,49
11082415274000,110824152,74000,MISSING PERSON,MISSING ADULT,Saturday,09/24/2011,11:00,TARAVAL,LOCATED,3200 Block of SAN JOSE AV,-122.459172646607,37.7082001648459,POINT (-122.459172646607 37.7082001648459),null,null,null,null,null,null,null,28588,null,null,null,null,null,null,null,null,null,null,null,null,null
4037801104134,040378011,04134,ASSAULT,BATTERY,Friday,12/12/2003,12:00,SOUTHERN,NONE,500 Block of I-80,-122.386667033903,37.7898821569191,POINT (-122.38666703390301 37.7898821569191),null,null,null,null,null,null,null,28856,null,null,null,null,null,null,null,null,null,null,null,null,null
4147669007025,041476690,07025,VEHICLE THEFT,STOLEN TRUCK,Thursday,12/30/2004,19:00,BAYVIEW,NONE,100 Block of KAREN CT,-122.407531316146,37.723291086507,POINT (-122.40753131614602 37.723291086507),91,2,2,25,10,3,7,309,28,null,null,null,null,null,null,1,null,null,null,null,91
16010127305073,160101273,05073,BURGLARY,"BURGLARY, UNLAWFUL ENTRY",Wednesday,02/03/2016,20:30,MISSION,"ARREST, BOOKED",2300 Block of 16TH ST,-122.40952991327822,37.76571843952822,POINT (-122.40952991327822 37.76571843952822),53,3,9,20,8,4,8,28853,19,null,null,null,3,null,null,2,null,null,3,null,53
17004924306243,170049243,06243,LARCENY/THEFT,PETTY THEFT FROM LOCKED AUTO,Wednesday,01/18/2017,18:20,TARAVAL,NONE,NORIEGA ST / SUNSET BL,-122.49522504431104,37.75341185338174,POINT (-122.49522504431104 37.75341185338174),39,10,7,35,1,8,3,56,35,null,null,null,null,null,null,1,null,null,null,null,39
16065828006244,160658280,06244,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Sunday,08/14/2016,21:00,TARAVAL,NONE,3200 Block of 20TH AV,-122.47564746078615,37.72852806274646,POINT (-122.47564746078615 37.72852806274647),41,10,8,16,1,8,4,64,14,null,null,null,null,null,null,2,null,null,null,null,41


In [0]:
## function to transform the date
from pyspark.sql.functions import to_date, to_timestamp, hour, month, year
sf_crime = sf_crime.withColumn('IncidntDate', to_date(sf_crime.Date, "MM/dd/yyyy"))
sf_crime = sf_crime.withColumn('Time', to_timestamp(sf_crime.Time, "HH:mm"))
sf_crime = sf_crime.withColumn('Hour', hour(sf_crime['Time']))
#df_opt1 = sf_crime.withColumn("DayOfWeek", date_format(sf_crime.Date, "EEEE"))
sf_crime = sf_crime.withColumn('Year', year(sf_crime['IncidntDate']))
sf_crime = sf_crime.withColumn('Month', month(sf_crime['IncidntDate']))

# dataset after time transformation
# create sql table
sf_crime.createOrReplaceTempView("sf_crime")
display(sf_crime)
sf_crime.cache()

PdId,IncidntNum,Incident Code,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,location,SF Find Neighborhoods 2 2,Current Police Districts 2 2,Current Supervisor Districts 2 2,Analysis Neighborhoods 2 2,DELETE - Fire Prevention Districts 2 2,DELETE - Police Districts 2 2,DELETE - Supervisor Districts 2 2,DELETE - Zip Codes 2 2,DELETE - Neighborhoods 2 2,DELETE - 2017 Fix It Zones 2 2,Civic Center Harm Reduction Project Boundary 2 2,Fix It Zones as of 2017-11-06 2 2,DELETE - HSOC Zones 2 2,Fix It Zones as of 2018-02-07 2 2,"CBD, BID and GBD Boundaries as of 2017 2 2","Areas of Vulnerability, 2016 2 2",Central Market/Tenderloin Boundary 2 2,Central Market/Tenderloin Boundary Polygon - Updated 2 2,HSOC Zones as of 2018-06-05 2 2,OWED Public Spaces 2 2,Neighborhoods 2,IncidntDate,Hour,Year,Month
3114751606302,031147516,06302,LARCENY/THEFT,PETTY THEFT FROM A BUILDING,Sunday,09/28/2003,1970-01-01T10:00:00.000+0000,SOUTHERN,NONE,1ST ST / BRANNAN ST,-120.5,90,POINT (-120.50000000000001 90),null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2003-09-28,10,2003,9
5069701104134,050697011,04134,ASSAULT,BATTERY,Wednesday,06/22/2005,1970-01-01T12:20:00.000+0000,NORTHERN,NONE,1200 Block of EDDY ST,-122.428223303176,37.7818959488603,POINT (-122.42822330317601 37.7818959488603),97,4,11,39,7,9,11,29490,41,null,null,null,null,null,null,2,null,null,null,null,97,2005-06-22,12,2005,6
6074729204104,060747292,04104,ASSAULT,ASSAULT,Saturday,07/15/2006,1970-01-01T00:55:00.000+0000,CENTRAL,NONE,1600 Block of POWELL ST,-122.410672425337,37.799788690123,POINT (-122.41067242533701 37.799788690123),106,6,3,6,3,1,10,308,4,null,null,null,null,null,null,2,null,null,null,null,106,2006-07-15,0,2006,7
7103536315201,071035363,15201,ASSAULT,STALKING,Tuesday,09/25/2007,1970-01-01T00:01:00.000+0000,TARAVAL,NONE,400 Block of ULLOA ST,-122.458226300605,37.7413616001449,POINT (-122.458226300605 37.7413616001449),49,10,8,41,1,8,4,59,40,null,null,null,null,null,null,1,null,null,null,null,49,2007-09-25,0,2007,9
11082415274000,110824152,74000,MISSING PERSON,MISSING ADULT,Saturday,09/24/2011,1970-01-01T11:00:00.000+0000,TARAVAL,LOCATED,3200 Block of SAN JOSE AV,-122.459172646607,37.7082001648459,POINT (-122.459172646607 37.7082001648459),null,null,null,null,null,null,null,28588,null,null,null,null,null,null,null,null,null,null,null,null,null,2011-09-24,11,2011,9
4037801104134,040378011,04134,ASSAULT,BATTERY,Friday,12/12/2003,1970-01-01T12:00:00.000+0000,SOUTHERN,NONE,500 Block of I-80,-122.386667033903,37.7898821569191,POINT (-122.38666703390301 37.7898821569191),null,null,null,null,null,null,null,28856,null,null,null,null,null,null,null,null,null,null,null,null,null,2003-12-12,12,2003,12
4147669007025,041476690,07025,VEHICLE THEFT,STOLEN TRUCK,Thursday,12/30/2004,1970-01-01T19:00:00.000+0000,BAYVIEW,NONE,100 Block of KAREN CT,-122.407531316146,37.723291086507,POINT (-122.40753131614602 37.723291086507),91,2,2,25,10,3,7,309,28,null,null,null,null,null,null,1,null,null,null,null,91,2004-12-30,19,2004,12
16010127305073,160101273,05073,BURGLARY,"BURGLARY, UNLAWFUL ENTRY",Wednesday,02/03/2016,1970-01-01T20:30:00.000+0000,MISSION,"ARREST, BOOKED",2300 Block of 16TH ST,-122.40952991327822,37.76571843952822,POINT (-122.40952991327822 37.76571843952822),53,3,9,20,8,4,8,28853,19,null,null,null,3,null,null,2,null,null,3,null,53,2016-02-03,20,2016,2
17004924306243,170049243,06243,LARCENY/THEFT,PETTY THEFT FROM LOCKED AUTO,Wednesday,01/18/2017,1970-01-01T18:20:00.000+0000,TARAVAL,NONE,NORIEGA ST / SUNSET BL,-122.49522504431104,37.75341185338174,POINT (-122.49522504431104 37.75341185338174),39,10,7,35,1,8,3,56,35,null,null,null,null,null,null,1,null,null,null,null,39,2017-01-18,18,2017,1
16065828006244,160658280,06244,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Sunday,08/14/2016,1970-01-01T21:00:00.000+0000,TARAVAL,NONE,3200 Block of 20TH AV,-122.47564746078615,37.72852806274646,POINT (-122.47564746078615 37.72852

Out[5]: DataFrame[PdId: string, IncidntNum: string, Incident Code: string, Category: string, Descript: string, DayOfWeek: string, Date: string, Time: timestamp, PdDistrict: string, Resolution: string, Address: string, X: string, Y: string, location: string, SF Find Neighborhoods 2 2: string, Current Police Districts 2 2: string, Current Supervisor Districts 2 2: string, Analysis Neighborhoods 2 2: string, DELETE - Fire Prevention Districts 2 2: string, DELETE - Police Districts 2 2: string, DELETE - Supervisor Districts 2 2: string, DELETE - Zip Codes 2 2: string, DELETE - Neighborhoods 2 2: string, DELETE - 2017 Fix It Zones 2 2: string, Civic Center Harm Reduction Project Boundary 2 2: string, Fix It Zones as of 2017-11-06 2 2: string, DELETE - HSOC Zones 2 2: string, Fix It Zones as of 2018-02-07 2 2: string, CBD, BID and GBD Boundaries as of 2017 2 2: string, Areas of Vulnerability, 2016 2 2: string, Central Market/Tenderloin Boundary 2 2: string, Central Market/Tenderloin Boundary Polygon - Updated 2 2: string, HSOC Zones as of 2018-06-05 2 2: string, OWED Public Spaces 2 2: string, Neighborhoods 2: string, IncidntDate: date, Hour: int, Year: int, Month: int]

####0. number of crimes by year

In [0]:
crime_yearly = spark.sql("select count(*) as yearly_crimes, Year from sf_crime group by 2 order by 2")
display(crime_yearly)

yearly_crimes,Year
142803,2003
142054,2004
137048,2005
131856,2006
131771,2007
135242,2008
134309,2009
127758,2010
126713,2011
135464,2012


#####Analysis
The line chart above shows the change of yearly crime counts from 2003 to 2018. 

From the line chart we can see that 2015 has the highest cirme rate while 2011 are around the lowest crime rate. The number of crimes descreased gradually from 2003 to 2011, where it reached the lowest in 2011. But from 2012 to 2017, the number of crimes rose rapidly. The crime rate was so high in 2015, which may be relevant to the Propostition 47 signed by the governor in the California referendum in 2014 that led to more theft and robbery crimes.

We also noticed that 2018 data is far less than other years, which is due to incomplete data with just first 4 months in 2018. So we will look at monthly count below.

In [0]:
#check the last record of 2018
last = spark.sql("select Date from sf_crime where Year = 2018 and Month = 5 order by Date desc limit 10")
display(last)

Date
05/15/2018
05/15/2018
05/15/2018
05/15/2018
05/15/2018
05/15/2018
05/15/2018
05/15/2018
05/15/2018
05/15/2018


In [0]:
crime_monthly = spark.sql("select count(*) as monthly_crimes, Year, Month from sf_crime group by 2,3 order by 2,3")
display(crime_monthly)

monthly_crimes,Year,Month
12366,2003,1
11377,2003,2
12400,2003,3
12251,2003,4
12041,2003,5
11451,2003,6
11889,2003,7
12390,2003,8
12213,2003,9
12475,2003,10


#####Analysis
As can be seen from above, 2018 only contains full records for the first 4 months and records only up to 15th in May. Therefore when comparing yearly difference or monthly difference for May-December, 2018 should be left out.

In 2013-2017, the crime rates are above the average of the 15-year range.

**Calendar effect:*

January, March, May, August and October have seen a rise on crimes from previous month, which may have something to do with one more day in calender. 

Feburary have the lowest crime numbers, which is due to the fact that it has 2 or 3 days fewer than other months.

#### 1. number of crimes for different category

In [0]:
crimeCategory = spark.sql("SELECT  category, COUNT(*) AS Count FROM sf_crime GROUP BY category ORDER BY Count DESC")
display(crimeCategory)

category,Count
LARCENY/THEFT,477975
OTHER OFFENSES,301874
NON-CRIMINAL,236928
ASSAULT,167042
VEHICLE THEFT,126228
DRUG/NARCOTIC,117821
VANDALISM,114718
WARRANTS,99821
BURGLARY,91067
SUSPICIOUS OCC,79087


#####Analysis
As can be seen from the bar chart, Larceny/Theft is the most committed crime at SF, which is about 60% more than the second crime 'Other offenses'. 

As mentioned before, Proposition 47, the 2014 ballot measure that reclassified nonviolent thefts as misdemeanors if the stolen goods are worth less than $950, had emboldened thieves.

Other offenses, non-criminal and assault are the second, third and forth of the most committed crimes respectively.

#### 2. the number of crimes for different district

In [0]:
crime_district = spark.sql("SELECT PdDistrict, count(*) as count from sf_crime group by 1 order by 2 desc")
display(crime_district)

PdDistrict,count
SOUTHERN,390692
MISSION,288985
NORTHERN,266435
CENTRAL,221923
BAYVIEW,205480
TENDERLOIN,186954
INGLESIDE,181092
TARAVAL,155461
PARK,119698
RICHMOND,112804


#####Analysis
As can be seen from the bar chart, most crimes are committed in Sounthern district. Mission and Northern district follows as the second and third place with crime counts similar to each other. Richmond and Park district have the lowest crime numbers. 

Therefore, when deciding where to live, I would suggest to consider those districts with lower crime rate such as Richmond, and avoiding areas close to Southern district. If go traveling or commuting to districts with higher crime rate, keep careful watch for potential danger in your surroundings .

#### 3. the number of crimes each Sunday at "SF downtown".   
SF downtown is defiend  via the range of spatial location. For example, you can use a rectangle to define the SF downtown, or you can define a cicle with center as well. 
San Francisco Latitude and longitude coordinates are: 37.773972, -122.431297. X and Y represents each. So we assume SF downtown spacial range: X (-122.4213,-122.4313), Y(37.7540,37.7740).

In [0]:
crime_Sun_SFdowntown = spark.sql("""
                                 with sunday_downtown_crime as(
                                 select substring(Date,1,5) as date,
                                        substring(Date,7) as year
                                 from sf_crime 
                                 where (DayOfWeek = 'Sunday' 
                                 and X <= -122.4213 
                                 and X >= -122.4313 
                                 and Y >= 37.7540 
                                 and Y <= 37.7740)
                                 )
                                 select year, date, count(*) as count
                                 from sunday_downtown_crime
                                 group by 1,2 
                                 order by 1,2
                                 """)

display(crime_Sun_SFdowntown)

year,date,count
2003,01/05,13
2003,01/12,20
2003,01/19,17
2003,01/26,13
2003,02/02,14
2003,02/09,22
2003,02/16,12
2003,02/23,14
2003,03/02,16
2003,03/09,8


#####Analysis
The bar chart above shows the history crime rate on a certain day of week(eg.Sunday) at a customized area (eg.SF downtown).
On 06/30, 2013, there was a peak of 54 crimes that Sunday at SF downtown. Police officers could further analyze that abnormal case to gain possible insights on the reason.

#### 4.the number of crime in each month of 2015, 2016, 2017, 2018.

In [0]:
monthly_crime = spark.sql("""
select Year, Month, count(*) as crime_number from sf_crime
where Year in (2015,2016,2017,2018) group by 1,2 order by 1,2
""")

display(monthly_crime)

Year,Month,crime_number
2015,1,13181
2015,2,11882
2015,3,13463
2015,4,12526
2015,5,13318
2015,6,12853
2015,7,12949
2015,8,13317
2015,9,12476
2015,10,12697


In [0]:
display(monthly_crime)

Year,Month,crime_number
2015,1,13181
2015,2,11882
2015,3,13463
2015,4,12526
2015,5,13318
2015,6,12853
2015,7,12949
2015,8,13317
2015,9,12476
2015,10,12697


#####Analysis

From the bar chart, we can see that in 2015-2017, January, March and May have most crimes.
August in 2015, October and December in 2016 and 2017 also have more crimes.

From the line chart, we can see a descreasing trend of crime rate from fourth quarter of 2015 to October 2016. It was not until the beginning of 2018 that crime rate significantly dropped in the first four months. This might suggest a good job done by the SFPD in 2018.

Why there's always more crimes in certain month besides the fact of more days on calendar? To answer this question, I took a look at crimes in December and January by date.

In [0]:
crime_Dec = spark.sql("select Year, substring(Date,1,5) as date, count(*) as crime_number from sf_crime where Year in (2015,2016,2017,2018) and Month in (12) group by 1,2 order by 1, 2")
display(crime_Dec)

Year,date,crime_number
2015,12/01,417
2015,12/02,363
2015,12/03,389
2015,12/04,423
2015,12/05,379
2015,12/06,358
2015,12/07,354
2015,12/08,386
2015,12/09,366
2015,12/10,387


In [0]:
crime_Jan = spark.sql("select Year, substring(Date,1,5) as date, count(*) as crime_number from sf_crime where Year in (2015,2016,2017,2018) and Month in (1) group by 1,2 order by 1, 2")
display(crime_Jan)

Year,date,crime_number
2015,01/01,538
2015,01/02,423
2015,01/03,436
2015,01/04,323
2015,01/05,447
2015,01/06,406
2015,01/07,391
2015,01/08,381
2015,01/09,489
2015,01/10,414


#####Analysis
From the line charts, it seems that days before New Year's Day always has the most crimes, but would drop immediatly after New Year's day. There's also the lowest crimes committed on the exact day of Chritmas. The period after Christmas until New Year's Day has witnessed a growing number of crimes.

Therefore, during these periods, the police should arrange more force, despite the fact that it should be holiday off. 

(Or is it possible that just because most police officers take breaks during this period and thus reduce the police force, that the criminals become more active and commit more crimes?)

#### 5.travel suggestion to visit SF
Analyze the number of crime w.r.t the hour in certian day like 2015/6/15, 2016/6/15, 2017/6/15.

In [0]:
hourly_crime_certainDay = spark.sql("""
select Date, 
Hour, 
count(*) as crime_number, 
Year
from sf_crime where Date like '06/15/%'
group by 4,1,2 order by 1,2
""")
display(hourly_crime_certainDay)

Date,Hour,crime_number,Year
06/15/2003,0,24,2003
06/15/2003,1,15,2003
06/15/2003,2,16,2003
06/15/2003,3,6,2003
06/15/2003,4,3,2003
06/15/2003,5,1,2003
06/15/2003,6,3,2003
06/15/2003,7,9,2003
06/15/2003,8,5,2003
06/15/2003,9,10,2003


In [0]:
#plot by total number
display(hourly_crime_certainDay)

Date,Hour,crime_number,Year
06/15/2003,0,24,2003
06/15/2003,1,15,2003
06/15/2003,2,16,2003
06/15/2003,3,6,2003
06/15/2003,4,3,2003
06/15/2003,5,1,2003
06/15/2003,6,3,2003
06/15/2003,7,9,2003
06/15/2003,8,5,2003
06/15/2003,9,10,2003


#####Analysis
The first line chart shows hourly crime rate grouped by years, and the second line chart shows the aggregate count across the 14 years from 2003 to 2017.

From 2003-2017 counts in total, we can see that on June 15 the highest crime number occurs around 12pm, and there's an increasing trend starting from 1pm and reached the second peak around 3pm. 

There're also fewer crimes in the morning than in the afternoon or evening. 

So tourists should better travel in the morning, and should be careful after the lunch time.

A further thinking: what type of crime should they watch out for? Is there a difference of most frequent types of crime across different hours in a day? 

Here I plot line chart for each hour showing hourly trends of different types of crime.

In [0]:
hourly_type_certainD = spark.sql("""
select Year, Hour, Date, Category, count(*) as Count
from sf_crime
where Date like '06/15/%'
group by 1,2,3,4 
order by 1,2,4""")
display(hourly_type_certainD)

Year,Hour,Date,Category,Count
2003,0,06/15/2003,ASSAULT,1
2003,0,06/15/2003,DISORDERLY CONDUCT,1
2003,0,06/15/2003,DRUNKENNESS,1
2003,0,06/15/2003,FRAUD,1
2003,0,06/15/2003,LARCENY/THEFT,5
2003,0,06/15/2003,NON-CRIMINAL,1
2003,0,06/15/2003,OTHER OFFENSES,5
2003,0,06/15/2003,SECONDARY CODES,1
2003,0,06/15/2003,STOLEN PROPERTY,1
2003,0,06/15/2003,VANDALISM,1


#####Analysis

From the above chart we can see that Larceny/Theft is the most frequent crime from around 11am to 23pm, while Other Offenses is the most frequent crime from 0am to 10am. Therefore, if tourists go out at noon, they should be most careful for their belongings.

Notice that not only tourists should be aware of such crimes, but also shop owners as well. Shoplifting epidemic has worsened over the years, which cause so much heavy burden on shop owner that more and more of the shops has closed because the scale of thefts had made business untenable. According to The New York Times, this spikes in organized retail crime is only witnessed in San Franciso but not other cities in the state, which implies the problem lies not in California law but mostlly on SFPD. The police should definitely provide support for the shop owners and other local business.

#### 6.advice to distribute the police
(1) Step1: Find out the top-3 danger disrict  
(2) Step2: find out the crime event w.r.t category and time (hour) from the result of step 1

In [0]:
#step1:
top3_dangerDistrict = spark.sql("select PdDistrict, count(*) as crime_number from sf_crime group by 1 order by 2 desc limit 3")
display(top3_dangerDistrict)

PdDistrict,crime_number
SOUTHERN,390692
MISSION,288985
NORTHERN,266435


In [0]:
#step2:
crime_event = spark.sql("""
select PdDistrict, Category, Hour , count(*) as count
from sf_crime 
where PdDistrict in ('SOUTHERN', 'MISSION', 'NORTHERN')
group by 1, 2, 3
order by 1, 2, 3
""")
display(crime_event)

PdDistrict,Category,Hour,count
MISSION,ARSON,0,35
MISSION,ARSON,1,23
MISSION,ARSON,2,28
MISSION,ARSON,3,36
MISSION,ARSON,4,27
MISSION,ARSON,5,26
MISSION,ARSON,6,12
MISSION,ARSON,7,8
MISSION,ARSON,8,7
MISSION,ARSON,9,14


#####Analysis

The top 3 dangerous districts are Southern, Mission and Northern.

(See the plot in Tableau)
https://public.tableau.com/profile/zishan.cheng#!/vizhome/sf_crimetop3districthourlycrimecategory/Sheet1?publish=yes

Here I assume that SF police force is arranged as a whole and each district also has some of its own force, so we need to distribute force across districts for SFPD, as well as across hours for each district's police office.

From the stacked bar chart in Tableau we can see that the highest number of crimes occurs at 6pm and 12pm. Additionally, the 5 most committed crimes in each district are larceny/theft, other offenses, non-criminal, assault and durg/narcotic, which accounts for over half of the crimes.


In the Southern district, the period from 11am to 12am has crime number above average of a day within that district. In Mission district, the period is from 11am to 12am. In Northern district, the period is from 12pm to 12am. So more police force should be distributed to these districts during these time periods.

#### 7. hints to adjust policy on different type of crime
For different category of crime, find the percentage of resolution.

In [0]:
%sql select distinct(resolution) as resolve from sf_crime

resolve
EXCEPTIONAL CLEARANCE
"ARREST, BOOKED"
PROSECUTED FOR LESSER OFFENSE
LOCATED
UNFOUNDED
DISTRICT ATTORNEY REFUSES TO PROSECUTE
PSYCHOPATHIC CASE
COMPLAINANT REFUSES TO PROSECUTE
"ARREST, CITED"
PROSECUTED BY OUTSIDE AGENCY


In [0]:
crime_total = spark.sql("select Category, count(*) as total from sf_crime group by Category order by 2 desc")
crime_total.createOrReplaceTempView("crime_total")
display(crime_total)
crime_total.cache()

Category,total
LARCENY/THEFT,477975
OTHER OFFENSES,301874
NON-CRIMINAL,236928
ASSAULT,167042
VEHICLE THEFT,126228
DRUG/NARCOTIC,117821
VANDALISM,114718
WARRANTS,99821
BURGLARY,91067
SUSPICIOUS OCC,79087


Out[47]: DataFrame[Category: string, total: bigint]

In [0]:
crime_res = spark.sql("select Category, count(*) as resolved from sf_crime where Resolution != 'NONE' group by 1")
crime_res.createOrReplaceTempView("crime_res")
display(crime_res)

Category,resolved
FRAUD,9291
SUICIDE,309
LIQUOR LAWS,2527
SECONDARY CODES,9542
MISSING PERSON,22638
OTHER OFFENSES,215059
DRIVING UNDER THE INFLUENCE,5335
WARRANTS,94339
ARSON,701
FORGERY/COUNTERFEITING,8546


In [0]:
resolve_rate = spark.sql("select a.Category, resolved, total, resolved/total as percentage from crime_total a left join crime_res b on a.Category = b.Category order by percentage desc")
display(resolve_rate)

Category,resolved,total,percentage
PROSTITUTION,15651,16501,0.9484879704260348
WARRANTS,94339,99821,0.9450816962362629
DRIVING UNDER THE INFLUENCE,5335,5652,0.9439136588818118
DRUG/NARCOTIC,107565,117821,0.912952699433887
LIQUOR LAWS,2527,2840,0.8897887323943662
LOITERING,2103,2402,0.8755203996669442
STOLEN PROPERTY,10011,11450,0.8743231441048035
DRUNKENNESS,8035,9760,0.8232581967213115
OTHER OFFENSES,215059,301874,0.7124131260062145
WEAPON LAWS,14934,21004,0.711007427156732


In [0]:
resolve = spark.sql("""
with resolve as (select Category, resolution, count(*) as tally
from sf_crime group by 1,2)
select a.Category, a.resolution, a.tally, b.total 
from resolve a 
left join (select * from crime_total) b
on a.Category = b.Category 
order by b.total desc, a.tally desc
""")
display(resolve)

Category,resolution,tally,total
LARCENY/THEFT,NONE,437927,477975
LARCENY/THEFT,"ARREST, BOOKED",25136,477975
LARCENY/THEFT,"ARREST, CITED",10463,477975
LARCENY/THEFT,NOT PROSECUTED,1600,477975
LARCENY/THEFT,UNFOUNDED,1221,477975
LARCENY/THEFT,COMPLAINANT REFUSES TO PROSECUTE,471,477975
LARCENY/THEFT,DISTRICT ATTORNEY REFUSES TO PROSECUTE,467,477975
LARCENY/THEFT,EXCEPTIONAL CLEARANCE,442,477975
LARCENY/THEFT,PROSECUTED BY OUTSIDE AGENCY,156,477975
LARCENY/THEFT,PSYCHOPATHIC CASE,47,477975


#####Analysis

From the bar chart above showing the percentage of resolution rate, we can see that prostitution, warrants and driving under the influence has the highest rates of resolution among all 37 categories, with resolution rates above 94%. 

However, from the other side of the bar chart, we can see that recovered vehicle, vehicle theft, larceny/theft has the lowest rats of resolution below 10%, followed by suspicious ooc, vandalism, with resoluation rates both below 15%. 

Why are so many of these types of crimes remain unresolved? Is it due to a lack of police capability or legal restrictions on these types? I believe this question should be answered by both parties. 

From the side of police department, I suggest them investigate into those unresolved cases, to see if there's a problem of laissez-faire attitute or ability. If there's a downplay on those cases, for example, taking it for granted that theft is hard to traced and thus making no effort to resolve it, then the police should rectify the situation. If there's a need for applying advanced techniques or skills, the department in charge should fix it. 

As for the policy makers, I suggest them investigate on the motivation behind those crime. If there's a difficulty for the police to resolve those cases (for either reasons), then it would be best to resolve it before it happens by adjusting relevant policies.

Notice that larceny/theft is the most committed crime, vandalism at 7th place, vehicle theft at 8th. There should be placed more emphasis on these types of crime with least resolution rate, either to enhance the police's technique/skills for solving such crimes or enforce more restrictions/punishments on those crimes.

The second pie chart demonstrates the resolution records of top 4 most committed crimes, so as to provide more directly visualization comparing crime number and resolution rate. 

As can be seen, the resolution rate is shockingly low for these most frequent crimes with more than 60% of them unresolved in three of the four most frequent crimes. So there's really an urgent need to improve this situation.

### Conclusion

####1. Motivation

When considering about whether to reside in a certain place or not, one of the most important factors is to research on the safety and crime rate of the area. Most people would prefer a safer place to live, especially for a mid- or long-term residence, or someone plan to run their own business. 

For every new UC Berkeley student coming from other area, he/she must have heard about the safety concerns around the campus: drug, homeless people, etc. As an international student from even another country, I remember the feeling of anxiety and uncertainty over the potential safety issue. Even though I have been here for over 1 year and completed my study safe and sound, I thought it would still be helpful to analyze the crime data, in an attempt  to understand what's going on around this place and how could residents/tourists/shop owners stay cautious and how could policies be further enhanced to improve the police force. 

I choose SF data because it is a world-famous metropolis with a population over 880K, and would also hopefully be the next place I were about to live.

####2. Methods

1) Data processing

  To begin with, in order to perform analysis on SF crime data from 2003-2018 of a large size more than 551MB, I utilized **Spark SQL** to load and manipulate the dataset. 

  The reason is that Spark offers a parallel method to process the data and thus leads to a much faster running speed than using Python pandas or Hadoop. **Spark SQL can directly read from multiple sources and ensures fast execution of existing Hive queries**. Moveover, **SQL can provide a easy way to load data without checking the many APIs from PySpark**, so it is a good way to get start. **After using Spark SQL to load data, one can use PySpark DataFrame format to get and transform the data into some representations that can be used to train machine learning model in the future.**
On deciding to use Spark SQL instead of PySpark DataFrame method, I compared the performance of both methods and noticed that SQL was faster.

2) Data visualization

  After data cleaning and processing, I utilized the built-in visualization tool in Datacricks by display() function for basic visualization need, as well as **Tableau** for more advanced demonstration purpose. I selected the features of interest such as categories, time, districts of crimes to visualize so that I could better understand and analyze the potential properties and relations among different features. 
  
  I also chose the most suitable form of plot from various types in order to fulfill the analysis need. For example, I used a **line chart and bar chart of month vs crime number and grouped lines by different year, so as to display clearly the variation of crime rate within one year and also across different years**. I also used **pie charts of percentage of resolution in each category of crime and sorted by crime frequency, showing clearly the resolution condition of the most frequent crime**. 
  
  When displaying the crime event w.r.t category and time (hour) in top 3 dangerous district, I plotted a **stacked bar chart combining with line chart and divided by districts, so that it is easy to compare hourly difference of number and type of crimes as well as that across different districts**.
  
How to select the best way for visualization makes a huge difference and is of great importance in conveying the insights from data analysis results.

####3. Analysis and insights
1. From the bar chart and line chart of count of crime in 2015-2018, I noticed a decreasing crime in 2018. From the charts of 15 years, I found an increasing trend of crime since 2011, but the first four months of 2018 so far has the lowest number of crime over the years.

2. From the bar chart of hour vs count of crime of different categories, and line plot of hour vs count of crime in a certain day, I found that most of crimes occur during lunch or in the afternoon in 14-22pm, and early morning has the least amount of crime, especially in 3-7am. So it is suggested to travel in the morning and assign more police in the evening.

3. From the bar chart and pie chart of categories vs resolution rate, Prostitution, Warrants and Driving under influence are most resolved, while Recovered vehicle, Vehicle theft and Larceny/theft are least resolved. Moveover, some categories with the lowest resolution rate are also the most frequently committed crimes, which should be fixed urgently and paid more effort and attention to by the police and policy makers.